In [1]:
import pickle as pkl
data_path = 'data.pkl'
with open(data_path, 'rb') as f:
    batch = pkl.load(f)

In [2]:
from models.CMA import LSTCWA, MultiDimAligner
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
D_TARGET   = 512
NUM_TOKENS = 64
WIN        = 64
STRIDE     = 32

aligner   = MultiDimAligner(D_TARGET).to(device)
compressor= LSTCWA(dim=D_TARGET, num_tokens=NUM_TOKENS,
                    win_size=WIN, stride=STRIDE).to(device)

# --------- 单 batch 推理示例 ---------
#compressed_batch = []
# 
# # 每张 WSI 输出一个 [L, D] token
tokens_per_backbone = []
for feats_list, coords_list, pad_list in zip(batch['imgs_list'],
                                            batch['coords_list'],
                                            batch['pad_mask_list']):
    """注意：
       - feats_list  : List[Tensor]，长度 = backbone 数
         每个 tensor 形状 [T_i, D_in_i]
       - coords_list : List[Tensor]，同上
       - pad_list    : List[BoolTensor]，同上
    """
    
    feats = feats.to(device)          # [T, D_in]
    coords= coords.to(device)
    pad   = pad.to(device)
    feats_ali = aligner(feats)        # → [T, D_TARGET]
    Z = compressor(feats_ali, coords, pad)  # [L, D_TARGET]
    tokens_per_backbone.append(Z)




In [25]:
for i in tokens_per_backbone:
    print(i.shape)

torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
